#IMPORT DATASETS AND LIBRARIES


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta 
import time
from os.path import exists


#Library

In [3]:
# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()
REST_API = 'https://api.pro.coinbase.com'
PRODUCTS = REST_API+'/products'

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

def get_single_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_15d_slope(df):
  df['15Close'] = df['Close'].shift(15)
  df['15Date'] = df['Date'].shift(15)
  df['Trend'] = (df['Close'] - df['15Close']) / 15
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):   
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product):
  start_date = (datetime.today() - timedelta(days=delta)).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':'86400'}
  response = connect(PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  return df_history

def get_coin_data_frames(time, product):
  df_raw = coinbase_json_to_df(time, product)
  df_btc_history = df_raw
  if len(df_btc_history.index) == 0:
    print("No data for ", product)
  df_btc_history['time'] = [datetime.fromtimestamp(x) for x in df_btc_history['time']]
  df_btc_history = df_btc_history.rename(columns={"time":"Date", "close":"Close", "volume":"Volume"})
  df_btc_history = sort_date(df_btc_history)
  df_btc_history = df_btc_history.drop(columns={"high", "low", "open"})
  df_btc_history = append_price_dif_(df_btc_history)
  df_btc_history = append_15d_slope(df_btc_history)
  df_btc_features = df_btc_history[["Close", "Volume", "Trend"]]
  df_history_scaled = sc.fit_transform(df_btc_features)
  return [df_btc_history, df_btc_features, df_history_scaled, df_raw]

def build_profit_estimate(predicted, df_btc_history):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def debug_prediction_frame(predicted, df_history, df_history_scaled):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Original"] = df_history_scaled[:,0]
  df_predicted_chart["Original-Target"] = df_history_scaled[:,2]
  df_predicted_chart["Target-Date"] = df_predicted_chart["Date"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted"] - df_predicted_chart["Original"]
  df_predicted_chart["Actual-Diff"] = df_predicted_chart["Original-Target"] - df_predicted_chart["Original"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["Close"] = df_history["Close"]
  df_predicted_chart["Target"] = df_history["Target"]
  df_predicted_chart["RealDiff"] = df_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_predicted_chart["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  return df_predicted_chart

def get_all_products():
  response = connect(PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

# Training

## Scale

In [4]:
model = None
model_path = "/content/drive/My Drive/model.h5"
file_exists = exists(model_path)
if file_exists:
  print("hello")
  model = keras.models.load_model(model_path)
else:
  print("nope")
  all_stocks_price_df = sort_date(pd.read_csv('/content/drive/My Drive/Colab Notebooks/stock.csv'))
  all_stocks_vol_df = sort_date(pd.read_csv("/content/drive/My Drive/Colab Notebooks/stock_volume.csv"))
  target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, "sp500")
  target_df = append_price_dif(target_df)
  target_df = append_15d_slope(target_df)
  features = target_df[["Close", "Volume", "Trend", "Target"]]
  scaled_features = scale_data(features)

hello


## Separate

In [5]:
if model == None:
  # Read stock prices data
  num_features = 3

  X = []
  y = []
  for i in range(0, len(target_df)):
      X.append(scaled_features [i][0:num_features])
      y.append(scaled_features [i][num_features])

  scaled_features[0]

## Actually, train

In [6]:
if model == None:
  X = np.asarray(X)
  y = np.asarray(y)

  # Split the data
  #split = int(0.7 * len(X))
  #X_train = X[:split]
  #y_train = y[:split]
  #X_test = X[split:]
  #y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))
  model = build_model(num_features, 1)
  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2
  )

# Backtest the coin in question

In [14]:
budget = 5000
[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(90, "SYN-USD")
predicted = model.predict(df_history_scaled).flatten()
[df_profit, profit] = build_profit_estimate(predicted, btc_history)
df_chart = debug_prediction_frame(predicted, btc_history, df_history_scaled)
interactive_plot(df_chart[["Date","Original", "Predicted"]], "Wtf")
print("Profit:", profit)
df_chart


3/3 [==============================] - 0s 13ms/step


Profit: 7293.534247027868


,Date,Predicted,Original,Original-Target,Target-Date,Predicted-Diff,Actual-Diff,Should-Trade,Close,Target,RealDiff,Percent,Profit
74,2022-10-10,0.861575,0.907258,0.444898,2022-10-11,-0.045683,-0.462360,False,1.101,1.090,-0.011,-0.009991,0.000000
73,2022-10-11,0.838264,0.892473,0.469388,2022-10-12,-0.054210,-0.423085,False,1.090,1.030,-0.060,-0.055046,0.000000
72,2022-10-12,0.775253,0.811828,0.420408,2022-10-13,-0.036575,-0.391420,False,1.030,0.966,-0.064,-0.062136,0.000000
71,2022-10-13,0.737268,0.725806,0.266667,2022-10-14,0.011462,-0.459140,True,0.966,0.963,-0.003,-0.003106,-15.527950
70,2022-10-14,0.697788,0.721774,0.212245,2022-10-15,-0.023987,-0.509529,False,0.963,0.973,0.010,0.010384,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-12-19,0.058583,0.053763,0.538776,2022-12-20,0.004819,0.485012,True,0.466,0.464,-0.002,-0.004292,-21.459227
3,2022-12-20,0.041266,0.051075,0.555102,2022-12-21,-0.009810,0.504027,False,0.464,0.426,-0.038,-0.081897,0.000000
2,2022-12-21,0.001072,0.000000,0.508844,2022-12-22,0.001072,0.508844,True,0.426,0.518,0.092,0.215962,1079.812207
1,2022-12-22,0.146635,0.123656,0.691156,2022-12-23,0.022979,0.567501,True,0.518,0.546,0.028,0.054054,270.270270


In [8]:
#interactive_plot(df_predicted_chart[["Date", "Predicted", "Original"]], "Original Vs Prediction")
#df_predicted_chart[["Date", "Close", "Target", "Percent", "RealDiff", "Bad-Trade", "Profit"]]


# Which coins are most profitable based on the above trading signals?

In [9]:
# download all known products and check who has the highest profit in 90 days
"""
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USD')]
df_products = df_products[df_products.trading_disabled == False]
df_products = df_products[df_products.cancel_only == False]
df_profit = pd.DataFrame();
df_profit["Product"] = [];
df_profit["Profit"] = [];
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)
    predicted = model.predict(df_scaled.flatten())
    [df_chart, profit] = build_profit_estimate(predicted, df_full)
    df_profit.loc[len(df_profit.index)] = [row.id, profit] 
  except Exception as inst:
    print("Error: ", inst)
  time.sleep(1)

df_profit
"""

'\n# Fetch the top 10 and see if they predict up\ndf_products = get_all_products()\ndf_products = df_products[df_products.id.str.endswith(\'USD\')]\ndf_products = df_products[df_products.trading_disabled == False]\ndf_products = df_products[df_products.cancel_only == False]\ndf_profit = pd.DataFrame();\ndf_profit["Product"] = [];\ndf_profit["Profit"] = [];\nfor index, row in df_products.iterrows():\n  try:\n    print("fetching: ", row.id)\n    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)\n    predicted = model.predict(df_scaled.flatten())\n    [df_chart, profit] = build_profit_estimate(predicted, df_full)\n    df_profit.loc[len(df_profit.index)] = [row.id, profit] \n  except Exception as inst:\n    print("Error: ", inst)\n  time.sleep(1)\n\ndf_profit\n'

# What has a buy indicator for tomorrow?

In [10]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USD')]
df_products = df_products[df_products.trading_disabled == False]
df_products = df_products[df_products.cancel_only == False]
df_trades = pd.DataFrame();
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(90, row.id)
    predicted = model.predict(npa_scaled).flatten()
    
    #convert to data frames that have the correct shape for being unscaled
    df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume", "Trend"])
    
    # I want to believe that scaling happens on a per column basis, we only care about
    # price here so we will dummy out volume and trend and use the scaler on it
    # this kinda sucks, if we add features we'll need to add them here for unscaling
    df_temp = pd.DataFrame(predicted, columns = ["Close"])
    df_temp["Volume"] = 0
    df_temp["Trend"] = 0
    
    # unscale them both
    df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume", "Trend"])
    df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume", "Trend"])
    
    # add predicted
    df_trade["Predicted"] = df_temp["Close"]
    df_trade = df_trade.tail(1)

    # add the product, derive a move and percent
    df_trade["Product"] = row.id;
    df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
    df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
   
    # we need to unscale the predicted values so that we have an entry and exit point
    # entry should be roughly close and exit should be roughly predicted

    # Stick this on the end of the main dataframe
    df_trades = df_trades.append(df_trade);
    
    #counter+=1
    #if counter > 5:
    #  break
  except Exception as inst:
    #raise inst
    print("Error: ", inst)
  time.sleep(1)
df_trades.reset_index()
df_trades = df_trades[df_trades['Move'] > 0] 





fetching:  AUCTION-USD
3/3 [==============================] - 0s 9ms/step
fetching:  MKR-USD
3/3 [==============================] - 0s 15ms/step
fetching:  ALICE-USD
3/3 [==============================] - 0s 12ms/step
fetching:  FET-USD
3/3 [==============================] - 0s 12ms/step
fetching:  BAND-USD
3/3 [==============================] - 0s 11ms/step
fetching:  SKL-USD
3/3 [==============================] - 0s 12ms/step
fetching:  CELR-USD
3/3 [==============================] - 0s 11ms/step
fetching:  INV-USD
3/3 [==============================] - 0s 12ms/step
fetching:  CTSI-USD
3/3 [==============================] - 0s 11ms/step
fetching:  CRPT-USD
3/3 [==============================] - 0s 9ms/step
fetching:  OXT-USD
3/3 [==============================] - 0s 11ms/step
fetching:  MTL-USD
3/3 [==============================] - 0s 11ms/step
fetching:  AIOZ-USD
3/3 [==============================] - 0s 13ms/step
fetching:  ANT-USD
1/1 [==============================] - 0s 23ms/st

In [11]:

df_trades

,Close,Volume,Trend,Predicted,Product,Move,Percent
74,0.7440,2.729604e+05,0.000000,0.753881,MTL-USD,0.009881,1.328111
74,5.6200,1.066572e+05,-0.110000,5.778822,LPT-USD,0.158822,2.826019
74,39.6500,4.249924e+04,-0.357333,40.184933,ZEC-USD,0.534933,1.349139
74,0.0588,8.631999e+06,-0.001747,0.058852,REN-USD,0.000052,0.088221
2,0.7076,1.111476e+05,-0.008987,0.707675,LIT-USD,0.000075,0.010563
1,34.7100,5.485430e+03,-0.726000,34.759183,EGLD-USD,0.049183,0.141697
74,0.0356,4.474943e+07,-0.000200,0.036742,LCX-USD,0.001142,3.206618
21,0.9270,8.394448e+05,-0.008067,0.951650,LDO-USD,0.024650,2.659078
74,0.0583,2.545128e+06,0.000073,0.058308,HOPR-USD,0.000008,0.012984
35,12.7100,3.701377e+03,-0.138000,12.768953,MSOL-USD,0.058953,0.463834


In [12]:
path = '/content/drive/My Drive/output.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_trades.to_csv(f)

In [13]:
 model.save(model_path)